# ML Model Serving with FastAPI

This notebook covers how to serve machine learning models with FastAPI:
- Loading sentence-transformers at startup (lifespan events)
- Creating embedding endpoints
- Batch prediction endpoint
- Model versioning
- Error handling for model failures

We'll build a production-ready embedding service using sentence-transformers.

## Setup

In [ ]:
from fastapi import FastAPI, HTTPException, status
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Any
from contextlib import asynccontextmanager
import time
from datetime import datetime
import numpy as np
from sentence_transformers import SentenceTransformer
import traceback

## 1. Lifespan Events - Loading Models at Startup

The lifespan context manager allows us to load models once when the app starts and clean up when it shuts down. This is more efficient than loading models on each request.

In [ ]:
# Global state for storing models
class ModelStore:
    """Store for ML models loaded at startup"""
    def __init__(self):
        self.models: Dict[str, Any] = {}
        self.model_metadata: Dict[str, Dict] = {}

model_store = ModelStore()

# Lifespan context manager
@asynccontextmanager
async def lifespan(app: FastAPI):
    """Load models on startup, cleanup on shutdown"""
    # Startup: Load models
    print("🚀 Loading ML models...")
    
    try:
        # Load small model for demos
        model_name = "all-MiniLM-L6-v2"
        print(f"Loading model: {model_name}...")
        model = SentenceTransformer(model_name)
        
        # Store model and metadata
        model_store.models["default"] = model
        model_store.model_metadata["default"] = {
            "name": model_name,
            "version": "1.0",
            "embedding_dim": model.get_sentence_embedding_dimension(),
            "max_seq_length": model.max_seq_length,
            "loaded_at": datetime.now().isoformat()
        }
        
        # Load a second model for demonstration
        model_name_v2 = "paraphrase-MiniLM-L3-v2"
        print(f"Loading model: {model_name_v2}...")
        model_v2 = SentenceTransformer(model_name_v2)
        
        model_store.models["v2"] = model_v2
        model_store.model_metadata["v2"] = {
            "name": model_name_v2,
            "version": "2.0",
            "embedding_dim": model_v2.get_sentence_embedding_dimension(),
            "max_seq_length": model_v2.max_seq_length,
            "loaded_at": datetime.now().isoformat()
        }
        
        print(f"✅ Successfully loaded {len(model_store.models)} models")
        
    except Exception as e:
        print(f"❌ Error loading models: {e}")
        raise
    
    # Yield control to the application
    yield
    
    # Shutdown: Cleanup
    print("🛑 Shutting down, cleaning up models...")
    model_store.models.clear()
    model_store.model_metadata.clear()
    print("✅ Cleanup complete")

# Create app with lifespan
app = FastAPI(
    title="ML Model Serving API",
    description="Serve sentence-transformer embeddings",
    version="1.0.0",
    lifespan=lifespan
)

print("FastAPI app with lifespan created!")

## 2. Pydantic Models for Requests and Responses

In [ ]:
# Request models
class EmbeddingRequest(BaseModel):
    """Request for single text embedding"""
    text: str = Field(..., min_length=1, max_length=5000, description="Text to embed")
    model_version: Optional[str] = Field("default", description="Model version to use")
    normalize: bool = Field(True, description="Normalize embeddings to unit length")
    
    class Config:
        schema_extra = {
            "example": {
                "text": "FastAPI is a modern web framework for building APIs",
                "model_version": "default",
                "normalize": True
            }
        }

class BatchEmbeddingRequest(BaseModel):
    """Request for batch text embeddings"""
    texts: List[str] = Field(..., min_items=1, max_items=100, description="Texts to embed")
    model_version: Optional[str] = Field("default", description="Model version to use")
    normalize: bool = Field(True, description="Normalize embeddings")
    batch_size: int = Field(32, ge=1, le=128, description="Batch size for encoding")
    
    @validator('texts')
    def validate_texts(cls, v):
        if any(len(text.strip()) == 0 for text in v):
            raise ValueError("All texts must be non-empty")
        return v

class SimilarityRequest(BaseModel):
    """Request for computing similarity between two texts"""
    text1: str = Field(..., min_length=1, max_length=5000)
    text2: str = Field(..., min_length=1, max_length=5000)
    model_version: Optional[str] = Field("default")

# Response models
class EmbeddingResponse(BaseModel):
    """Response with embedding vector"""
    embedding: List[float] = Field(..., description="Embedding vector")
    dimension: int = Field(..., description="Embedding dimension")
    model_version: str = Field(..., description="Model version used")
    processing_time_ms: float = Field(..., description="Processing time in milliseconds")

class BatchEmbeddingResponse(BaseModel):
    """Response with batch embeddings"""
    embeddings: List[List[float]] = Field(..., description="List of embedding vectors")
    count: int = Field(..., description="Number of embeddings")
    dimension: int = Field(..., description="Embedding dimension")
    model_version: str = Field(..., description="Model version used")
    processing_time_ms: float = Field(..., description="Processing time in milliseconds")

class SimilarityResponse(BaseModel):
    """Response with similarity score"""
    similarity: float = Field(..., ge=-1.0, le=1.0, description="Cosine similarity (-1 to 1)")
    text1_preview: str = Field(..., description="Preview of first text")
    text2_preview: str = Field(..., description="Preview of second text")
    model_version: str

class ModelInfo(BaseModel):
    """Model metadata"""
    name: str
    version: str
    embedding_dim: int
    max_seq_length: int
    loaded_at: str

print("Pydantic models defined!")

## 3. Model Information Endpoints

In [ ]:
@app.get("/")
async def root():
    """Root endpoint with service info"""
    return {
        "service": "ML Model Serving API",
        "version": "1.0.0",
        "models_loaded": len(model_store.models),
        "available_models": list(model_store.models.keys()),
        "endpoints": [
            "/models",
            "/embed",
            "/embed/batch",
            "/similarity"
        ]
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    models_loaded = len(model_store.models) > 0
    return {
        "status": "healthy" if models_loaded else "unhealthy",
        "models_loaded": len(model_store.models),
        "timestamp": datetime.now().isoformat()
    }

@app.get("/models", response_model=Dict[str, ModelInfo])
async def list_models():
    """List all available models and their metadata"""
    if not model_store.models:
        raise HTTPException(
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail="No models loaded"
        )
    return model_store.model_metadata

@app.get("/models/{model_version}", response_model=ModelInfo)
async def get_model_info(model_version: str):
    """Get information about a specific model"""
    if model_version not in model_store.models:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Model '{model_version}' not found. Available: {list(model_store.models.keys())}"
        )
    return model_store.model_metadata[model_version]

print("Model info endpoints added!")

## 4. Embedding Endpoints

In [ ]:
@app.post("/embed", response_model=EmbeddingResponse)
async def create_embedding(request: EmbeddingRequest):
    """Create embedding for a single text"""
    start_time = time.time()
    
    try:
        # Validate model version
        if request.model_version not in model_store.models:
            raise HTTPException(
                status_code=status.HTTP_404_NOT_FOUND,
                detail=f"Model '{request.model_version}' not found. Available: {list(model_store.models.keys())}"
            )
        
        # Get model
        model = model_store.models[request.model_version]
        
        # Create embedding
        embedding = model.encode(
            request.text,
            normalize_embeddings=request.normalize,
            show_progress_bar=False
        )
        
        # Convert to list
        embedding_list = embedding.tolist()
        
        processing_time = (time.time() - start_time) * 1000
        
        return {
            "embedding": embedding_list,
            "dimension": len(embedding_list),
            "model_version": request.model_version,
            "processing_time_ms": round(processing_time, 2)
        }
        
    except HTTPException:
        raise
    except Exception as e:
        print(f"Error creating embedding: {e}")
        print(traceback.format_exc())
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Error creating embedding: {str(e)}"
        )

@app.post("/embed/batch", response_model=BatchEmbeddingResponse)
async def create_batch_embeddings(request: BatchEmbeddingRequest):
    """Create embeddings for multiple texts in batch"""
    start_time = time.time()
    
    try:
        # Validate model version
        if request.model_version not in model_store.models:
            raise HTTPException(
                status_code=status.HTTP_404_NOT_FOUND,
                detail=f"Model '{request.model_version}' not found"
            )
        
        # Get model
        model = model_store.models[request.model_version]
        
        # Create embeddings in batch (much faster than one-by-one)
        embeddings = model.encode(
            request.texts,
            batch_size=request.batch_size,
            normalize_embeddings=request.normalize,
            show_progress_bar=False
        )
        
        # Convert to list of lists
        embeddings_list = embeddings.tolist()
        
        processing_time = (time.time() - start_time) * 1000
        
        return {
            "embeddings": embeddings_list,
            "count": len(embeddings_list),
            "dimension": len(embeddings_list[0]) if embeddings_list else 0,
            "model_version": request.model_version,
            "processing_time_ms": round(processing_time, 2)
        }
        
    except HTTPException:
        raise
    except Exception as e:
        print(f"Error creating batch embeddings: {e}")
        print(traceback.format_exc())
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Error creating batch embeddings: {str(e)}"
        )

print("Embedding endpoints added!")

## 5. Similarity Endpoint

In [ ]:
@app.post("/similarity", response_model=SimilarityResponse)
async def compute_similarity(request: SimilarityRequest):
    """Compute cosine similarity between two texts"""
    try:
        # Validate model version
        if request.model_version not in model_store.models:
            raise HTTPException(
                status_code=status.HTTP_404_NOT_FOUND,
                detail=f"Model '{request.model_version}' not found"
            )
        
        # Get model
        model = model_store.models[request.model_version]
        
        # Create embeddings for both texts
        embeddings = model.encode(
            [request.text1, request.text2],
            normalize_embeddings=True,  # Normalized for cosine similarity
            show_progress_bar=False
        )
        
        # Compute cosine similarity (dot product of normalized vectors)
        similarity = float(np.dot(embeddings[0], embeddings[1]))
        
        return {
            "similarity": similarity,
            "text1_preview": request.text1[:100] + ("..." if len(request.text1) > 100 else ""),
            "text2_preview": request.text2[:100] + ("..." if len(request.text2) > 100 else ""),
            "model_version": request.model_version
        }
        
    except HTTPException:
        raise
    except Exception as e:
        print(f"Error computing similarity: {e}")
        print(traceback.format_exc())
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Error computing similarity: {str(e)}"
        )

print("Similarity endpoint added!")

## 6. Running the Server

In [ ]:
# Create TestClient with context manager to trigger lifespan events
print("Creating TestClient and triggering model loading...")
print("(This may take a moment to download and load models)\n")

## 7. Testing the API

In [ ]:
# Test 1: Root endpoint
# Use 'with' statement to ensure lifespan events run
with TestClient(app) as client:
    response = client.get("/")
    print("Test 1 - Service info:")
    print(f"Response: {response.json()}")
    print()

In [ ]:
# Test 2: List available models
with TestClient(app) as client:
    response = client.get("/models")
    print("Test 2 - Available models:")
    models = response.json()
    for version, info in models.items():
        print(f"\nModel: {version}")
        print(f"  Name: {info['name']}")
        print(f"  Embedding dimension: {info['embedding_dim']}")
        print(f"  Max sequence length: {info['max_seq_length']}")
    print()

In [ ]:
# Test 3: Create single embedding
with TestClient(app) as client:
    request_data = {
        "text": "FastAPI is a modern web framework for building APIs with Python",
        "model_version": "default",
        "normalize": True
    }
    response = client.post("/embed", json=request_data)
    result = response.json()

    print("Test 3 - Single embedding:")
    print(f"Model version: {result['model_version']}")
    print(f"Embedding dimension: {result['dimension']}")
    print(f"Processing time: {result['processing_time_ms']:.2f}ms")
    print(f"First 5 values: {result['embedding'][:5]}")
    print()

In [ ]:
# Test 4: Batch embeddings
with TestClient(app) as client:
    texts = [
        "Machine learning is a subset of artificial intelligence",
        "FastAPI is built on Starlette and Pydantic",
        "Python is a popular programming language",
        "Natural language processing analyzes text data",
        "Deep learning uses neural networks"
    ]

    request_data = {
        "texts": texts,
        "model_version": "default",
        "normalize": True,
        "batch_size": 32
    }

    response = client.post("/embed/batch", json=request_data)
    result = response.json()

    print("Test 4 - Batch embeddings:")
    print(f"Number of texts: {result['count']}")
    print(f"Embedding dimension: {result['dimension']}")
    print(f"Processing time: {result['processing_time_ms']:.2f}ms")
    print(f"Time per text: {result['processing_time_ms'] / result['count']:.2f}ms")
    print()

In [ ]:
# Test 5: Similarity computation
with TestClient(app) as client:
    # Similar texts
    request_data = {
        "text1": "Machine learning is a type of artificial intelligence",
        "text2": "Artificial intelligence includes machine learning",
        "model_version": "default"
    }
    response = client.post("/similarity", json=request_data)
    result = response.json()

    print("Test 5a - Similar texts:")
    print(f"Similarity: {result['similarity']:.4f}")
    print(f"Text 1: {result['text1_preview']}")
    print(f"Text 2: {result['text2_preview']}")
    print()

    # Dissimilar texts
    request_data = {
        "text1": "I love pizza and pasta",
        "text2": "Python is a programming language",
        "model_version": "default"
    }
    response = client.post("/similarity", json=request_data)
    result = response.json()

    print("Test 5b - Dissimilar texts:")
    print(f"Similarity: {result['similarity']:.4f}")
    print()

In [ ]:
# Test 6: Different model versions
with TestClient(app) as client:
    text = "FastAPI makes building APIs easy"

    for model_version in ["default", "v2"]:
        request_data = {
            "text": text,
            "model_version": model_version
        }
        response = client.post("/embed", json=request_data)
        result = response.json()
        
        print(f"Model: {model_version}")
        print(f"  Dimension: {result['dimension']}")
        print(f"  Processing time: {result['processing_time_ms']:.2f}ms")
    print()

In [ ]:
# Test 7: Error handling - invalid model
with TestClient(app) as client:
    request_data = {
        "text": "Test text",
        "model_version": "nonexistent"
    }
    response = client.post("/embed", json=request_data)

    print("Test 7 - Error handling (invalid model):")
    print(f"Status code: {response.status_code}")
    print(f"Error: {response.json()['detail']}")
    print()

In [ ]:
# Test 8: Validation error - empty text
with TestClient(app) as client:
    request_data = {
        "text": "",  # Empty text
        "model_version": "default"
    }
    response = client.post("/embed", json=request_data)

    print("Test 8 - Validation error (empty text):")
    print(f"Status code: {response.status_code}")
    print(f"Error details: {response.json()}")
    print()

## 8. Performance Comparison: Single vs Batch

In [ ]:
with TestClient(app) as client:
    texts = [
        "Machine learning models can be served via APIs",
        "FastAPI provides automatic documentation",
        "Batch processing is more efficient than single requests",
        "Sentence transformers create text embeddings",
        "Python is widely used in machine learning",
        "Deep learning requires substantial computational resources",
        "Natural language processing analyzes human language",
        "Vector embeddings capture semantic meaning",
        "API endpoints can handle batch operations",
        "Model versioning enables gradual rollouts"
    ]

    # Method 1: Individual requests
    start_time = time.time()
    for text in texts:
        client.post("/embed", json={"text": text})
    individual_time = (time.time() - start_time) * 1000

    # Method 2: Batch request
    start_time = time.time()
    client.post("/embed/batch", json={"texts": texts})
    batch_time = (time.time() - start_time) * 1000

    print("Performance comparison (10 texts):")
    print(f"Individual requests: {individual_time:.2f}ms")
    print(f"Batch request: {batch_time:.2f}ms")
    print(f"Speedup: {individual_time / batch_time:.2f}x faster with batching")
    print(f"\n💡 Key insight: Batch processing is much more efficient!")

## 9. Key Takeaways

### What we learned:

1. **Lifespan Events**:
   - Use `@asynccontextmanager` for startup/shutdown logic
   - Load models once at startup (not on each request)
   - Store models in a global state object
   - Clean up resources on shutdown

2. **Model Serving Best Practices**:
   - Load models at startup for better performance
   - Support multiple model versions
   - Provide model metadata endpoints
   - Return processing time in responses

3. **Batch Processing**:
   - Batch endpoints are much more efficient
   - Use appropriate batch sizes
   - Set reasonable limits (max 100 texts in example)
   - Track processing time to monitor performance

4. **Error Handling**:
   - Validate model versions exist
   - Catch and handle model errors gracefully
   - Return appropriate HTTP status codes
   - Provide helpful error messages

5. **Model Versioning**:
   - Support multiple model versions simultaneously
   - Allow clients to specify which version to use
   - Track metadata for each model
   - Enable A/B testing and gradual rollouts

### Production Considerations:
- Add request rate limiting
- Implement request queuing for high load
- Monitor model performance metrics
- Add model caching strategies
- Implement health checks for models
- Add logging and tracing

### Next steps:
- In the next notebook, we'll explore async operations and background tasks
- We'll cover file uploads, streaming responses, and concurrent request handling

In [ ]:
print(f"\n🎉 Congratulations! You've completed ML Model Serving!\n")
print(f"Your embedding service is ready for testing!")
print(f"In production, you would run: uvicorn app:app --host 0.0.0.0 --port 8000")